In [ ]:
import os
import pickle

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

from utils import train, evaluate, make_kaggle_submission
from plots import plot_learning_curves, plot_confusion_matrix
from mydatasets import calculate_num_features, VisitSequenceWithLabelDataset, visit_collate_fn
from mymodels import MyVariableRNN

torch.manual_seed(0)
if torch.cuda.is_available():
	torch.cuda.manual_seed(0)

# Set a correct path to the data files that you preprocessed
PATH_TRAIN_SEQS = "../data/mortality/processed/mortality.seqs.train"
PATH_TRAIN_LABELS = "../data/mortality/processed/mortality.labels.train"
PATH_VALID_SEQS = "../data/mortality/processed/mortality.seqs.validation"
PATH_VALID_LABELS = "../data/mortality/processed/mortality.labels.validation"
PATH_TEST_SEQS = "../data/mortality/processed/mortality.seqs.test"
PATH_TEST_LABELS = "../data/mortality/processed/mortality.labels.test"
PATH_TEST_IDS = "../data/mortality/processed/mortality.ids.test"
PATH_OUTPUT = "../output/mortality/"

NUM_EPOCHS = 10
BATCH_SIZE = 64
USE_CUDA = False  # Set 'True' if you want to use GPU
NUM_WORKERS = 0

# Data loading
print('===> Loading entire datasets')
train_seqs = pickle.load(open(PATH_TRAIN_SEQS, 'rb'))
train_labels = pickle.load(open(PATH_TRAIN_LABELS, 'rb'))
valid_seqs = pickle.load(open(PATH_VALID_SEQS, 'rb'))
valid_labels = pickle.load(open(PATH_VALID_LABELS, 'rb'))
test_seqs = pickle.load(open(PATH_TEST_SEQS, 'rb'))
test_labels = pickle.load(open(PATH_TEST_LABELS, 'rb'))

num_features = calculate_num_features(train_seqs)

train_dataset = VisitSequenceWithLabelDataset(train_seqs, train_labels, num_features)
valid_dataset = VisitSequenceWithLabelDataset(valid_seqs, valid_labels, num_features)
test_dataset = VisitSequenceWithLabelDataset(test_seqs, test_labels, num_features)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=visit_collate_fn, num_workers=NUM_WORKERS)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=visit_collate_fn, num_workers=NUM_WORKERS)
# batch_size for the test set should be 1 to avoid sorting each mini-batch which breaks the connection with patient IDs
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False, collate_fn=visit_collate_fn, num_workers=NUM_WORKERS)

model = MyVariableRNN(num_features)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

device = torch.device("cuda" if torch.cuda.is_available() and USE_CUDA else "cpu")
model.to(device)
criterion.to(device)

best_val_acc = 0.0
train_losses, train_accuracies = [], []
valid_losses, valid_accuracies = [], []
for epoch in range(NUM_EPOCHS):
	train_loss, train_accuracy = train(model, device, train_loader, criterion, optimizer, epoch)
	valid_loss, valid_accuracy, valid_results = evaluate(model, device, valid_loader, criterion)

	train_losses.append(train_loss)
	valid_losses.append(valid_loss)

	train_accuracies.append(train_accuracy)
	valid_accuracies.append(valid_accuracy)

	is_best = valid_accuracy > best_val_acc  # let's keep the model that has the best accuracy, but you can also use another metric.
	if is_best:
		best_val_acc = valid_accuracy
		torch.save(model, os.path.join(PATH_OUTPUT, "MyVariableRNN.pth"))

best_model = torch.load(os.path.join(PATH_OUTPUT, "MyVariableRNN.pth"))
# TODO: For your report, try to make plots similar to those in the previous task.
# TODO: You may use the validation set in case you cannot use the test set.

plot_learning_curves(train_losses, valid_losses, train_accuracies, valid_accuracies)
test_loss, test_accuracy, test_results = evaluate(best_model, device, valid_loader, criterion)
class_names = ['Mortality', 'Not Mortality']
plot_confusion_matrix(test_results, class_names)

===> Loading entire datasets


/Users/nabeelnauman/Documents/OMSCS/Big-Data-GaTech-Assignments/homework5/code/mymodels.py:115: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc1.weight)
/Users/nabeelnauman/Documents/OMSCS/Big-Data-GaTech-Assignments/homework5/code/mymodels.py:126: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc2.weight)


Epoch: [0][0/83]	Time 0.145 (0.145)	Data 0.013 (0.013)	Loss 0.6681 (0.6681)	Accuracy 68.000 (68.000)
Epoch: [0][10/83]	Time 0.019 (0.033)	Data 0.002 (0.005)	Loss 0.6105 (0.6494)	Accuracy 70.000 (64.182)
Epoch: [0][20/83]	Time 0.029 (0.029)	Data 0.007 (0.005)	Loss 0.5895 (0.6366)	Accuracy 67.000 (64.857)
Epoch: [0][30/83]	Time 0.020 (0.028)	Data 0.003 (0.005)	Loss 0.6415 (0.6296)	Accuracy 64.000 (65.258)
Epoch: [0][40/83]	Time 0.030 (0.027)	Data 0.006 (0.005)	Loss 0.5973 (0.6209)	Accuracy 64.000 (65.390)
Epoch: [0][50/83]	Time 0.025 (0.027)	Data 0.006 (0.005)	Loss 0.6763 (0.6131)	Accuracy 60.000 (65.961)
Epoch: [0][60/83]	Time 0.019 (0.026)	Data 0.004 (0.005)	Loss 0.5866 (0.6076)	Accuracy 62.000 (66.410)
Epoch: [0][70/83]	Time 0.020 (0.025)	Data 0.003 (0.005)	Loss 0.4828 (0.5969)	Accuracy 75.000 (67.408)
Epoch: [0][80/83]	Time 0.018 (0.025)	Data 0.003 (0.005)	Loss 0.5226 (0.5884)	Accuracy 71.000 (68.062)
Test: [0/12]	Time 0.007 (0.007)	Loss 0.5589 (0.5589)	Accuracy 70.000 (70.000)
Test:

In [ ]:
model

In [ ]:
test_accuracy